# Dressmaker - Easy

In [1]:
import $ivy.`org.apache.spark::spark-sql:3.4.0`

import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.spark._
import org.apache.spark.sql._
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions.Window

val spark = {
    NotebookSparkSession.builder()
    .progress(false)
    .appName("app17-1")
    // .master("spark://192.168.31.31:7077")
    .master("local[*]")
    .config("spark.sql.warehouse.dir", 
            "hdfs://192.168.31.31:9000/user/hive/warehouse") 
    .config("spark.cores.max", "4") 
    .config("spark.executor.instances", "1") 
    .config("spark.executor.cores", "2") 
    .config("spark.executor.memory", "10g") 
    .config("spark.shuffle.service.enabled", "false") 
    .config("spark.dynamicAllocation.enabled", "false") 
    .config("spark.sql.catalogImplementation", "hive")
    .config("spark.sql.repl.eagerEval.enabled", "true")
    .config("spark.driver.allowMultipleContexts", "true")
    .getOrCreate()
}

Loading spark-stubs, spark-hive
Adding Hive conf dir /opt/hive/conf to classpath
Creating SparkSession


SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See https://www.slf4j.org/codes.html#noProviders for further details.


Spark UI

import $ivy.$                                  


import org.apache.log4j.{Level, Logger}

import org.apache.spark._

import org.apache.spark.sql._

import org.apache.spark.sql.types._

import org.apache.spark.sql.functions._

import org.apache.spark.sql.expressions.Window


spark: SparkSession = org.apache.spark.sql.SparkSession@4b52afa9

In [2]:
import spark.implicits._
import spark.sqlContext.implicits._
def sc = spark.sparkContext
val hiveCxt = new org.apache.spark.sql.hive.HiveContext(sc)

import spark.implicits._

import spark.sqlContext.implicits._

defined function sc
hiveCxt: sql.hive.HiveContext = org.apache.spark.sql.hive.HiveContext@b3993c9

In [3]:
// Credit to Aivean
implicit class RichDF(val ds:DataFrame) {
    def showHTML(limit: Int = 50, truncate: Int = 100) = {
        import xml.Utility.escape
        val data = ds.take(limit)
        val header = ds.schema.fieldNames.toSeq        
        val rows: Seq[Seq[String]] = data.map { row =>
          row.toSeq.map {cell =>
            val str = cell match {
              case null => "null"
              case binary: Array[Byte] => binary.map("%02X".format(_)).mkString("[", " ", "]")
              case array: Array[_] => array.mkString("[", ", ", "]")
              case seq: Seq[_] => seq.mkString("[", ", ", "]")
              case _ => cell.toString
            }
            if (truncate > 0 && str.length > truncate) {
              // do not show ellipses for strings shorter than 4 characters.
              if (truncate < 4) str.substring(0, truncate)
              else str.substring(0, truncate - 3) + "..."
            } else {
              str
            }
          }: Seq[String]
        }
    publish.html(s""" <table>
                <tr>
                 ${header.map(h => s"<th>${escape(h)}</th>").mkString}
                </tr>
                ${rows.map {row =>
                  s"<tr>${row.map{c => s"<td>${escape(c)}</td>" }.mkString}</tr>"
                }.mkString}
            </table>
        """)
    }
}

defined class RichDF

In [4]:
val jmcust = hiveCxt.table("sqlzoo.jmcust")
val dressmaker = hiveCxt.table("sqlzoo.dressmaker")
val dress_order = hiveCxt.table("sqlzoo.dress_order")
val construction = hiveCxt.table("sqlzoo.construction")
val quantities = hiveCxt.table("sqlzoo.quantities")
val order_line = hiveCxt.table("sqlzoo.order_line")
val garment = hiveCxt.table("sqlzoo.garment")
val material = hiveCxt.table("sqlzoo.material")

jmcust: DataFrame = [c_no: int, c_name: string ... 2 more fields]
dressmaker: DataFrame = [d_no: int, d_name: string ... 2 more fields]
dress_order: DataFrame = [order_no: int, cust_no: int ... 2 more fields]
construction: DataFrame = [maker: int, order_ref: int ... 3 more fields]
quantities: DataFrame = [style_q: int, size_q: int ... 1 more field]
order_line: DataFrame = [order_ref: int, line_no: int ... 3 more fields]
garment: DataFrame = [style_no: int, description: string ... 2 more fields]
material: DataFrame = [material_no: int, fabric: string ... 3 more fields]

## 1.
List the post code, order number, order date and garment descriptions for all items associated with Ms Brown.

In [5]:
val t1 = (jmcust.filter(trim(jmcust("c_name")) === "Ms Brown")
      .join(dress_order, (jmcust("c_no") === dress_order("cust_no")))
      .join(order_line, (dress_order("order_no") === order_line("order_ref")))
      .join(garment, (order_line("ol_style") === garment("style_no")))
      .withColumnRenamed("c_post_code", "post_code")
      .select("post_code", "order_no", "order_date", "description"))
val t2 = (dressmaker.filter(trim(dressmaker("d_name")) === "Ms Brown")
      .join(construction, (dressmaker("d_no") === construction("maker")))
      .join(dress_order, (construction("order_ref") === dress_order("order_no")))
      .join(order_line, (dress_order("order_no") === order_line("order_ref")))
      .join(garment, (order_line("ol_style") === garment("style_no")))
      .withColumnRenamed("d_post_code", "post_code")
      .select("post_code", "order_no", "order_date", "description"))
t1.union(t2).showHTML()

post_code,order_no,order_date,description
B2 5AB,9,2002-02-27,Shorts
B2 5AB,7,2002-02-21,Suntop
B2 5AB,7,2002-02-21,Sundress
B2 5AB,7,2002-02-21,Short Skirt


t1: DataFrame = [post_code: string, order_no: int ... 2 more fields]
t2: DataFrame = [post_code: string, order_no: int ... 2 more fields]

## 2.
List the customer name, postal information, order date and order number of all orders that have been completed.

In [6]:
(jmcust.join(dress_order.filter(dress_order("completed") === "Y"),
             (jmcust("c_no") === dress_order("cust_no")))
 .join(order_line, (dress_order("order_no") === order_line("order_ref")))
 .select("c_name", "c_post_code", "order_date", "order_no")
 .showHTML())

c_name,c_post_code,order_date,order_no
Ms White,E24 8PQ,2002-02-03,5
Ms White,E24 8PQ,2002-02-03,5
Ms White,E24 8PQ,2002-02-03,5
Mr Brass,FG24 9NM,2002-02-02,4
Mr Brass,FG24 9NM,2002-02-02,4
Ms Muir,H2 7CV,2002-01-20,3
Dr Green,SJ4 4WE,2002-01-11,2
Dr Green,SJ4 4WE,2002-01-11,2
Dr Green,SJ4 4WE,2002-01-11,2
Mrs Peacock,DD6 9NM,2002-01-10,1


## 3.
Which garments have been made or are being made from "red abstract" or "blue abstract" coloured materials.

In [7]:
(garment.join(order_line, (garment("style_no") === order_line("ol_style")))
 .join(material.filter(lower(trim(material("colour"))).isin(
     List("red abstract", "blue abstract"): _*)),
     (order_line("ol_material") === material("material_no")))
 .select("style_no", "description", "colour")
 .showHTML())

style_no,description,colour
1,Trousers,Red Abstract
2,Long Skirt,Blue Abstract
2,Long Skirt,Red Abstract
4,Short Skirt,Blue Abstract
5,Sundress,Red Abstract


## 4.
How many garments has each dressmaker constructed? You should give the number of garments and the name and postal information of each dressmaker.

In [8]:
(dressmaker.join(construction, (dressmaker("d_no") === construction("maker")))
 .join(dress_order, (construction("order_ref") === dress_order("order_no")))
 .join(order_line, (dress_order("order_no") === order_line("order_ref")))
 .join(garment, (order_line("ol_style") === garment("style_no")))
 .groupBy("d_no", "d_name", "d_post_code")
 .agg(count("line_no").alias("cnt"))
 .showHTML())

d_no,d_name,d_post_code,cnt
3,Mr Needles,E12 6LG,12
1,Mrs Hem,A12 6BC,14
4,Ms Sew,EF7 9KL,12
5,Mr Seam,H45 7YH,12
6,Mr Taylor,SH6 9RT,7
7,Miss Pins,B4 9BL,11
2,Miss Stitch,DF4 7HJ,20


## 5.
Give the names of those dressmakers who have finished items made from silk for completed orders.

In [9]:
(dressmaker.join(construction, (dressmaker("d_no") === construction("maker")))
 .join(dress_order.filter(dress_order("completed") === "Y"), 
       (construction("order_ref") === dress_order("order_no")))
 .join(order_line, (dress_order("order_no") === order_line("order_ref")))
 .join(garment, (order_line("ol_style") === garment("style_no")))
 .join(material.filter(lower(trim(material("fabric"))) === "silk"), 
        (order_line("ol_material") === material("material_no")))
 .select("d_name")
 .distinct()
 .showHTML())

d_name
Ms Sew
Miss Stitch
Mrs Hem
Miss Pins
Mr Needles


In [10]:
spark.stop()